In [ ]:
from data import MultiMolGraphDataset
from torch_geometric.loader import DataLoader
import torch
import torchmetrics
from torchmetrics import MeanAbsoluteError
import random
import pandas as pd
import numpy as np
from siamesepairwise import SiameseDimeNet
from torch.optim.lr_scheduler import CosineAnnealingLR
from schedulers import CosineRestartsDecay
import os
import argparse
import torch.nn.functional as F
from torch_geometric.data import Data
from tqdm import tqdm  # for progress bars
from loss_utils import cosine_angle_loss, AngularErrorMetric


device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


In [2]:
sdf_path = '/home/calvin/code/chemprop_phd_customised/habnet/data/processed/sdf_data'
target_data = '/home/calvin/code/chemprop_phd_customised/habnet/data/processed/target_data/target_data.csv'

In [ ]:
# Read in target_data with pandas and then select 'psi_1_dihedral' column and convert the dihedral angles to sin and cos 

# 1 & 2. Load your data
target_df = pd.read_csv(target_data)

# Drop -10 rows in the dihedral angle column
target_df = target_df[target_df['psi_1_dihedral'] != -10]

# 3. Extract the dihedral angles
angles = target_df['psi_1_dihedral']

# 3.1 Remove any -10 rows in the dihedral angle column
angles = angles[angles != -10]
# 3.2 Remove any NaN values
angles = angles.dropna()

# 4. If angles are in degrees, convert to radians
angles_rad = np.deg2rad(angles)

# 5. Compute sin & cos and assign
target_df['psi_1_dihedral_sin'] = np.sin(angles_rad)
target_df['psi_1_dihedral_cos'] = np.cos(angles_rad)

# Optional: inspect
print(target_df[['psi_1_dihedral', 'psi_1_dihedral_sin', 'psi_1_dihedral_cos']].head())



# 6. Save the modified DataFrame back to CSV
target_data = '/home/calvin/code/chemprop_phd_customised/habnet/data/processed/target_data/target_data_sin_cos.csv'
target_df.to_csv(target_data, index=False)

   psi_1_dihedral  psi_1_dihedral_sin  psi_1_dihedral_cos
0      313.550300           -0.724770            0.688991
2      292.580884           -0.923338            0.383987
3      233.012473           -0.798766           -0.601641
4       34.468778            0.565957            0.824435
5      202.755671           -0.386802           -0.922163


In [4]:
from data import EquiMultiMolDataset
import os
# 1) Prepare a root folder for PyG to use
root = 'data/equi_dataset'
os.makedirs(root, exist_ok=True)
# 2) Point to your SDF files folder and the CSV of targets
sdf_folder     = '/home/calvin/code/chemprop_phd_customised/habnet/data/processed/sdf_data'      # ← replace with your actual path
target_csv     = '/home/calvin/code/chemprop_phd_customised/habnet/data/processed/target_data/target_data_sin_cos.csv'
input_types    = ['r1h', 'r2h']       # ← the values of the SDF “type” property you want
target_columns = ['psi_1_dihedral_sin', 'psi_1_dihedral_cos']    # ← your CSV columns for sin & cos

# 3) Instantiate the dataset (this will auto-run process() if needed)
equi_dataset = EquiMultiMolDataset(
    root=root,
    sdf_folder=sdf_folder,
    target_csv=target_csv,
    input_type=input_types,
    target_columns=target_columns,
    keep_hs=True,
    sanitize=False,
    force_reload=True
)
                                   

Processing 1810 SDF files from ['/home/calvin/code/chemprop_phd_customised/habnet/data/processed/sdf_data']
RXN IDs available in CSV: ['rmg_rxn_86', 'rxn_618', 'rxn_1833', 'rmg_rxn_182', 'rxn_1240', 'rmg_rxn_157', 'rxn_563', 'rmg_rxn_521', 'kfir_rxn_13503', 'rxn_1423']...
  • kfir_rxn_10218.sdf → found types ['r1h', 'r2h']
  • kfir_rxn_10391.sdf → found types ['r1h', 'r2h']
  • kfir_rxn_11197.sdf → found types ['r1h', 'r2h']
  • kfir_rxn_11663.sdf → found types ['r1h', 'r2h']
  • kfir_rxn_11797.sdf → found types ['r1h', 'r2h']
  • kfir_rxn_11915.sdf → found types ['r1h', 'r2h']
  • kfir_rxn_12959.sdf → found types ['r1h', 'r2h']
  • kfir_rxn_13108.sdf → found types ['r1h', 'r2h']
  • kfir_rxn_1315.sdf → found types ['r1h', 'r2h']
  • kfir_rxn_13503.sdf → found types ['r1h', 'r2h']
  • kfir_rxn_13651.sdf → found types ['r1h', 'r2h']
  • kfir_rxn_13712.sdf → found types ['r1h', 'r2h']
  • kfir_rxn_13897.sdf → found types ['r1h', 'r2h']
  • kfir_rxn_2.sdf → found types ['r1h', 'r2h']
  • 

Processing...


  • rmg_rxn_15515.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15516.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15517.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15521.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15524.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15526.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15531.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15538.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15539.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15540.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15541.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15545.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15565.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15566.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15567.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15575.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15576.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15577.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15579.sdf → found types ['r1h', 'r2h']
  • rmg_rxn_15580.sdf → found t

Done!


In [5]:
equi_dataset

EquiMultiMolDataset(1696)

In [6]:
from dimenet import DimeNetPPEncoder

encoder = DimeNetPPEncoder(hidden_channels=512,
                           dropout=0.2, num_blocks=6,
                           num_spherical=8, num_radial=7,
                           cutoff=6.0)
encoder = encoder.to(device)

In [7]:
# import dataloader from pytorch - not torch_geometric
from torch.utils.data import DataLoader
from data import siamese_collate
train_loader = DataLoader(equi_dataset, batch_size=1, shuffle=True, collate_fn=siamese_collate)

In [8]:
from torch_geometric.loader import DataLoader

# We need to split randomly into train/valid/test sets
from torch.utils.data import random_split

train_size = int(0.8 * len(equi_dataset))
valid_size = int(0.1 * len(equi_dataset))
test_size = len(equi_dataset) - train_size - valid_size
train_dataset, valid_dataset, test_dataset = random_split(equi_dataset, [train_size, valid_size, test_size])
# Create DataLoaders for each dataset
train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True,
    follow_batch=['z_s', 'z_t']
)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=32,
    shuffle=True,
    follow_batch=['z_s', 'z_t']
)
test_loader = DataLoader(
    test_dataset,
    batch_size=32,
    shuffle=True,
    follow_batch=['z_s', 'z_t']
)


In [ ]:

def train_epoch(model, loader, optimizer, loss_fn, metric_fn):
    model.train()
    total_loss = 0.0
    total_err  = 0.0
    n_samples  = 0

    for batch in tqdm(loader, desc="Training", leave=False):
        batch = batch.to(device)
        optimizer.zero_grad()

        # forward + loss
        out   = model(batch)              # [B,2]
        loss  = loss_fn(out, batch.y)     # scalar
        loss.backward()
        optimizer.step()

        # metric: mean abs angle error (degrees)
        err = metric_fn(out, batch.y)     # scalar tensor

        bsize = batch.y.size(0)
        total_loss += loss.item() * bsize
        total_err  += err.item()  * bsize
        n_samples  += bsize

    return total_loss / n_samples, total_err / n_samples

def eval_epoch(model, loader, loss_fn, metric_fn):
    model.eval()
    total_loss = 0.0
    total_err  = 0.0
    n_samples  = 0

    with torch.no_grad():
        for batch in tqdm(loader, desc="Validation", leave=False):
            batch = batch.to(device)
            out   = model(batch)
            loss  = loss_fn(out, batch.y)
            err   = metric_fn(out, batch.y)

            bsize = batch.y.size(0)
            total_loss += loss.item() * bsize
            total_err  += err.item()  * bsize
            n_samples  += bsize

    return total_loss / n_samples, total_err / n_samples


In [ ]:
import argparse
import json
import os
from datetime import datetime
import yaml

def save_experiment(config: dict,
                    metrics: dict,
                    out_dir: str = "experiments"):
    """
    Dump one JSON per run, containing both hyper-parameters and
    your final validation metrics.
    """
    os.makedirs(out_dir, exist_ok=True)
    # timestamped filename
    ts = datetime.now().strftime("%Y%m%d-%H%M%S")
    fname = os.path.join(out_dir, f"run_{ts}.json")
    payload = {
        "config":  config,
        "metrics": metrics,
    }
    with open(fname, "w") as f:
        json.dump(payload, f, indent=2)
    print(f"👉 Saved experiment to {fname}")

# --- when you start training, assemble your hyper‐params: ---
config = {
    "encoder_hidden":    256,
    "num_blocks":        4,
    "num_spherical":     7,
    "num_radial":        6,
    "cutoff":            5.0,
    "fusion":            "diff-prod",   # or "cat"
    "dropout":           0.1,
    "batch_size":        32,
    "lr":                1e-4,
    "weight_decay":      1e-5,
    "scheduler":         "CosineRestartsDecay(T0=20,decay=0.6)",
    "loss":              "cosine_angle_loss",
    "epochs":            num_epochs
}

# ... inside your training loop you track best_val_error (degrees) etc ...
metrics = {
    "best_val_loss":   best_val_loss,
    "best_val_error":  best_val_err,   # e.g. AngularErrorMetric in ° 
    "final_val_loss":  va_loss,
    "final_val_error": va_err,
}

# at the end of training:
save_experiment(config, metrics)

# ─── parse config ────────────────────────────────────────────────────
parser = argparse.ArgumentParser()
parser.add_argument("--config", type=str, required=True,
                    help="Path to YAML config file")
args = parser.parse_args()

with open(args.config, "r") as f:
    cfg = yaml.safe_load(f)

# ─── make experiment directory ────────────────────────────────────────
ts = datetime.now().strftime("%Y%m%d-%H%M%S")
exp_name = f"dimenetpp_{ts}"
out_dir = os.path.join(cfg["logging"]["output_root"], exp_name)
os.makedirs(out_dir, exist_ok=True)


with open(os.path.join(out_dir, "used_config.yaml"), "w") as f:
    yaml.safe_dump(cfg, f)

In [ ]:
# Set Seed
seed = cfg["training"]["seed"]
seed_everything(seed)

# Build the model
enc_cfg = cfg["model"]
encoder = DimeNetPPEncoder(
    hidden_channels=enc_cfg["hidden_channels"],
    num_blocks=enc_cfg["num_blocks"],
    num_spherical=enc_cfg["num_spherical"],
    num_radial=enc_cfg["num_radial"],
    cutoff=enc_cfg["cutoff"],
    dropout=enc_cfg["dropout"],
).to(device)

model = SiameseDimeNet(
    encoder=encoder,
    fusion=cfg["model"]["fusion"],
    dropout=cfg["model"]["dropout"],
).to(device)

# Build the optimizer & Scheduler
tr_cfg = cfg["training"]
optimizer = torch.optim.AdamW(
    model.parameters(),
    lr=tr_cfg["lr"],
    weight_decay=tr_cfg["weight_decay"],
)
sch_cfg = tr_cfg["scheduler"]
if sch_cfg["name"] == "CosineRestartsDecay":
    scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
        optimizer,
        T_0=sch_cfg["T_0"],
        T_mult=sch_cfg["T_mult"],
        eta_min=sch_cfg["eta_min"],
    )
else:
    raise ValueError(f"Unknown scheduler: {sch_cfg['name']}")
# ─── build the loss function ──────────────────────────────────────────
loss_fn = cosine_angle_loss
metric_fn = AngularErrorMetric(
    in_degrees=True,)
# Build data loaders

## Randomly split the dataset into train/valid/test sets
train_size = int(0.8 * len(equi_dataset))
valid_size = int(0.1 * len(equi_dataset))
test_size = len(equi_dataset) - train_size - valid_size
train_dataset, valid_dataset, test_dataset = random_split(equi_dataset, [train_size, valid_size, test_size])
# Create DataLoaders for each dataset
train_loader = DataLoader(
    train_dataset,
    batch_size=cfg["training"]["batch_size"],
    shuffle=True,
    follow_batch=['z_s', 'z_t']
)
valid_loader = DataLoader(
    valid_dataset,
    batch_size=cfg["training"]["batch_size"],
    shuffle=True,
    follow_batch=['z_s', 'z_t']
)
test_loader = DataLoader(
    test_dataset,
    batch_size=cfg["training"]["batch_size"],
    shuffle=True,
    follow_batch=['z_s', 'z_t']
)





EquiDataBatch(y=[16, 2], z_s=[128], z_s_batch=[128], z_s_ptr=[17], pos_s=[128, 3], z_t=[165], z_t_batch=[165], z_t_ptr=[17], pos_t=[165, 3], id=[16])


In [10]:

model = SiameseDimeNet(
    encoder=encoder)
model = model.to(device)
num_epochs  = 200
learning_rate = 1e-4
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=1e-5)
# scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
#     optimizer, mode='min', factor=0.5, patience=5, verbose=True
# )
#scheduler = CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=1e-6)
scheduler = CosineRestartsDecay(
    optimizer,
    T_0     = 20,
    T_mult  = 2,
    eta_min = 1e-4,
    decay   = 0.3) 


import torch.nn.functional as F

best_val_loss = float('inf')

loss_fn = cosine_angle_loss
metric_fn   = AngularErrorMetric(in_degrees=True)

# 3) Training loop

for epoch in range(1, num_epochs + 1):
    tr_loss, tr_err = train_epoch(model, train_loader, optimizer, loss_fn, metric_fn)
    va_loss, va_err = eval_epoch(model, valid_loader, loss_fn, metric_fn)

    # Step the scheduler once per epoch
    # If your scheduler wants the epoch number, do: scheduler.step(epoch)
    # Otherwise just:
    scheduler.step()

    lr = scheduler.get_last_lr()[0]
    print(f"Epoch {epoch:02d} | "
          f"Train L={tr_loss:.4f}, Err={tr_err:.1f}° | "
          f"Val   L={va_loss:.4f}, Err={va_err:.1f}° | "
          f"LR={lr:.2e}")

    # (Optional) save best‐model checkpoint
    if va_loss < best_val_loss:
        best_val_loss = va_loss
        torch.save(model.state_dict(), 'best_dimenet_model.pt')
        print(" ↳ New best model saved!")

Training:   0%|          | 0/85 [00:00<?, ?it/s]/home/calvin/miniforge3/envs/dmpnn_rocm/lib/python3.10/site-packages/torch/nn/modules/linear.py:125: UserWarning: Attempting to use hipBLASLt on an unsupported architecture! Overriding blas backend to hipblas (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:310.)
  return F.linear(input, self.weight, self.bias)


Epoch 01 | Train L=0.7585, Err=71.9° | Val   L=0.7352, Err=70.6° | LR=1.00e-04
 ↳ New best model saved!


Epoch 02 | Train L=0.7192, Err=69.3° | Val   L=0.7241, Err=69.7° | LR=1.00e-04
 ↳ New best model saved!


Epoch 03 | Train L=0.7097, Err=68.6° | Val   L=0.7324, Err=69.9° | LR=1.00e-04


Epoch 04 | Train L=0.7068, Err=68.4° | Val   L=0.7066, Err=67.8° | LR=1.00e-04
 ↳ New best model saved!


...

Epoch 82 | Train L=0.3650, Err=42.4° | Val   L=0.4900, Err=52.0° | LR=2.50e-05


Epoch 83 | Train L=0.3626, Err=42.1° | Val   L=0.5101, Err=53.5° | LR=2.63e-05


Epoch 84 | Train L=0.3937, Err=44.4° | Val   L=0.5032, Err=53.6° | LR=2.78e-05


Epoch 85 | Train L=0.3702, Err=42.7° | Val   L=0.4891, Err=52.5° | LR=2.92e-05


KeyboardInterrupt: 

In [ ]:
batch = next(iter(train_loader))
print(batch)

EquiDataBatch(y=[32, 2], z_s=[334], z_s_batch=[334], z_s_ptr=[33], pos_s=[334, 3], z_t=[341], z_t_batch=[341], z_t_ptr=[33], pos_t=[341, 3], id=[32])


In [16]:
torch.cuda.empty_cache()
print("CUDA memory cleared")
# Print CUDA memory summary


CUDA memory cleared


In [15]:
print("CUDA memory summary:")
print(torch.cuda.memory_summary(device=None, abbreviated=False))

CUDA memory summary:
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 10        |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      | 419315 KiB |   6020 MiB |  63448 GiB |  63448 GiB |
|       from large pool |  79195 KiB |   5742 MiB |  60360 GiB |  60360 GiB |
|       from small pool | 340120 KiB |    415 MiB |   3088 GiB |   3087 GiB |
|---------------------------------------------------------------------------|
| Active memory         | 419315 KiB |   6020 MiB |  63448 GiB |  63448 GiB |
|       from large pool |  79195 KiB |   57

In [ ]:
batch.y

tensor([[ 9.9498e-01,  1.0010e-01],
        [-8.8480e-01,  4.6598e-01],
        [-7.0213e-01, -7.1205e-01],
        [-9.9791e-01,  6.4688e-02],
        [ 7.9861e-02, -9.9681e-01],
        [ 3.9891e-01, -9.1699e-01],
        [-4.3020e-02,  9.9907e-01],
        [-1.6591e-01,  9.8614e-01],
        [ 1.8761e-01, -9.8224e-01],
        [-9.6775e-01, -2.5191e-01],
        [-7.0285e-01,  7.1134e-01],
        [ 2.6074e-01,  9.6541e-01],
        [-5.3904e-04,  1.0000e+00],
        [-1.3275e-01, -9.9115e-01],
        [-4.9938e-04,  1.0000e+00],
        [-1.6487e-01, -9.8632e-01],
        [ 8.2383e-01,  5.6684e-01],
        [ 9.9917e-01,  4.0807e-02],
        [ 1.1098e-01,  9.9382e-01],
        [-9.3298e-01, -3.5993e-01],
        [ 2.6962e-01,  9.6297e-01],
        [-4.5483e-03, -9.9999e-01],
        [-3.3019e-01,  9.4391e-01],
        [ 3.2036e-01, -9.4730e-01],
        [ 9.9998e-01,  6.6410e-03],
        [ 1.0000e+00, -1.0429e-03],
        [ 3.4122e-01, -9.3999e-01],
        [-9.9093e-01, -1.344